## step 1
mount your google drive and get link the video to the colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## step
to access the video

In [2]:
video_dir = "/content/drive/MyDrive/deepFakeDetector/testProject"


## step
extract the frames from the video and upload those frames in a colab file


In [3]:
import cv2
import os

def extract_frames(video_path, output_dir, label):
    os.makedirs(os.path.join(output_dir, label), exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret or count > 10:  # extract only 10 frames to keep it small
            break
        frame_path = os.path.join(output_dir, label, f"{os.path.basename(video_path)}_frame{count}.jpg")
        cv2.imwrite(frame_path, frame)
        count += 1
    cap.release()

# Process all videos
root_video_dir = "/content/drive/MyDrive/deepFakeDetector/projectTest"
output_dir = "/deepFakeDetecor/frames"

for folder in os.listdir(root_video_dir):
    label = "real" if "real" in folder.lower() else "fake"
    folder_path = os.path.join(root_video_dir, folder)
    if os.path.isdir(folder_path):
        for video_file in os.listdir(folder_path):
            video_path = os.path.join(folder_path, video_file)
            extract_frames(video_path, output_dir, label)



## step
import pytorch and , setup dataloader and train a deepfake detection model


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import os

# Path to the frames saved in the google colab
data_dir = "/deepFakeDetecor/frames"

# Transform the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load dataset
dataset = datasets.ImageFolder(data_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

class_names = dataset.classes
print("Classes:", class_names)

# Load pretrained model
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # 2 classes: real, fake

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
for epoch in range(5):  # You can increase this
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Epoch {epoch+1}, Loss: {running_loss:.4f}, Accuracy: {100 * correct / total:.2f}%")

# Save model
torch.save(model.state_dict(), "deepfake_model.pth")
print("Model saved as deepfake_model.pth")


Classes: ['fake', 'real']


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 144MB/s]


Epoch 1, Loss: 46.8536, Accuracy: 70.05%
Epoch 2, Loss: 30.8436, Accuracy: 63.64%
Epoch 3, Loss: 29.9129, Accuracy: 65.24%
Epoch 4, Loss: 25.4372, Accuracy: 68.98%
Epoch 5, Loss: 25.9523, Accuracy: 72.73%
Model saved as deepfake_model.pth


## step
- Prediction Function : this is used after training to test any single frame.

In [5]:
from PIL import Image

def predict_image(image_path):
    img = Image.open(image_path).convert('RGB')
    img_tensor = transform(img).unsqueeze(0).to(device)
    model.eval()
    with torch.no_grad():
        output = model(img_tensor)
        _, predicted = torch.max(output, 1)
        return class_names[predicted.item()]

# Example usage
print(predict_image("/deepFakeDetecor/frames/real/id0_0004.mp4_frame6.jpg"))
# real image for real id5 frame10 doesnt detect well , the model thinks its a fake image


real
